In [1]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue, set_start_method
from datetime import datetime

In [2]:
def get_output_file(state_file: str)->str:
    return state_file + "-out"

In [3]:
def reexecute_and_train(state_file: str, cell: str, model_name: str):
    import dill
    import torch
    output_file = get_output_file(state_file)
    # load state from the users notebook
    dill.load_module(state_file)
    # train the model
    exec(cell)
    # save the model
    torch.save(globals()[model_name].state_dict(), output_file)

In [4]:
def execute_in_subprocess(state_file, cell, model_name, stdout_listener):
    from contextlib import redirect_stdout
    # set_start_method("spawn")
    p = Process(target=reexecute_and_train, args=(state_file, cell, model_name))
    import sys
    class DummyStdout:
        def __init__(self) -> None:
            self.conn = stdout_listener.accept()
        def write(self, text):
            self.conn.send(text)

    dummy_stout = DummyStdout()
    with redirect_stdout(dummy_stout):
        p.start()
        p.join()
    dummy_stout.conn.send("DONE")

In [5]:
from multiprocessing.connection import Listener
from multiprocessing import Process, Queue

address = ('localhost', 6001)     # family is deduced to be 'AF_INET'
listener = Listener(address, authkey=b'secret password')

stdout_listener = Listener(('localhost', 6002), authkey=b'secret password')

try:
    while True:
        conn = listener.accept()
        state_file = conn.recv()
        cell = conn.recv()
        model_name=conn.recv()
        filename = execute_in_subprocess(state_file, cell, model_name, stdout_listener)
        conn.send(get_output_file(state_file))
finally:
    listener.close()